In [10]:
import numpy as np
from scipy import interp
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from gensim import models
from sklearn import metrics
from scipy import spatial

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

j = 0
for i in range(3):

    LOAD_MODEL = 'master(validation{:d}).model'.format(i)
    HIGH_PATH = './train{:d}/high_train{:d}'.format(i, i)
    MID_PATH = './train{:d}/middle_train{:d}'.format(i, i)

    exec(open('./parseValidFiles.py').read())

    model = models.Doc2Vec.load(LOAD_MODEL)

    labels = []
    values = []
    file_num = 0

    for root, dirs, files in os.walk(MID_PATH):    
            file_num += len(files)
    for root, dirs, files in os.walk(MID_PATH):    
        for i in range(len(vali_sentences)):
            sum_sim = 0
            vali_vector = model.infer_vector(vali_sentences[i].words)
            for file in files:
                docpath = os.path.join(root, file)     
                docu_vector = model.docvecs[docpath]
                doc_dis = 1 - spatial.distance.cosine(vali_vector, docu_vector) 
                sum_sim += doc_dis
            ave_sim = sum_sim / file_num
            if 'middle' in vali_sentences[i].tags[0]:
                label = 1
                labels.append(label)
            else:
                label = 2
                labels.append(label)
            values.append(ave_sim)
        
    y = np.array(labels)
    scores = np.array(values)
    
    print(y)
    print(scores)

    fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=1, drop_intermediate=False)
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    auc = metrics.auc(fpr, tpr)
    aucs.append(auc)

    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (j, auc))
    print(j)
    j += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Luck', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)
std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Risk Determination - Receiver operating characteristic Curve')
plt.legend(loc="lower right")
plt.show()

plt.savefig('middle.png')

/home/spsv06/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


[2 2 2 2 1 1 1 1 1 1 1 1 2 2 2]
[0.5357079  0.52092558 0.62962435 0.57610887 0.51650544 0.64689512
 0.57802673 0.52714236 0.52201654 0.55712029 0.57574906 0.63307577
 0.55818036 0.66341205 0.54503038]
0
[2 2 2 2 2 2 2 1 1 1 1 1 1 1 1]
[0.60631227 0.63263251 0.6163332  0.50955103 0.57876424 0.52270835
 0.62998881 0.59827092 0.54406254 0.63743872 0.51926898 0.63603629
 0.52024795 0.6444896  0.55267377]
1
[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2]
[0.58950739 0.5971276  0.51203478 0.65927101 0.61738868 0.62455333
 0.56190322 0.65992387 0.53927152 0.62139802 0.59573678 0.61700937
 0.50377662 0.61048416 0.38340259]
2
